In [8]:
import duckdb
import pandas as pd
from pandas.tseries.offsets import MonthBegin
import numpy as np
import xarray as xr
import logging
from datetime import datetime, timedelta
import duckdb
import itables

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True,
)
log = logging.getLogger()

itables.init_notebook_mode(all_interactive=True)

# 1. Add Weather Data

## 1.1 Choosing Weather Dataset

[google-research/arco-era5](https://github.com/google-research/arco-era5)

To characterize weather conditions for each BlueBikes trip, we use the **ARCO-ERA5 0.25° Hourly Analysis-Ready Dataset**, a public, cloud-optimized version of the ERA5 global reanalysis.  
ARCO-ERA5 provides hourly, spatially consistent, and quality-controlled atmospheric fields on a regular 0.25° latitude–longitude grid (~28 km spacing). Because the dataset is already regridded, chunked, and stored in Zarr format, it can be accessed efficiently for large-scale analytics and is well-suited for mobility studies that require weather information joined to millions of trip records.

For every trip, we extract weather variables at the trip’s **start time** (rounded to the nearest hour) and **nearest grid cell**, ensuring spatially and temporally consistent meteorological conditions.


In [2]:
# open the Zarr store directly (read-only)
ds = xr.open_zarr(
    # "gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr",
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks={'time': 48}, 
    consolidated=True,
)
print(ds)

<xarray.Dataset> Size: 4PB
Dimensions:                                                          (
                                                                      time: 1323648,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * time                                                             (time) datetime64[ns] 11MB ...
  * latitude                                                         (latitude) float32 3kB ...
  * longitude                                                        (longitude) float32 6kB ...
  * level                                                            (level) int64 296B ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB dask.array<chunksize=(48, 721, 1

In [3]:
ds.latitude

<xarray.DataArray 'latitude' (latitude: 721)> Size: 3kB
array([ 90.  ,  89.75,  89.5 , ..., -89.5 , -89.75, -90.  ],
      shape=(721,), dtype=float32)
Coordinates:
  * latitude  (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
Attributes:
    long_name:  latitude
    units:      degrees_north

In [4]:
ds.longitude
# Note: longitude is from 0 to 360 (not -180 to 180)

<xarray.DataArray 'longitude' (longitude: 1440)> Size: 6kB
array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
       3.5975e+02], shape=(1440,), dtype=float32)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    long_name:  longitude
    units:      degrees_east

In [5]:
str(list(ds.data_vars))

"['100m_u_component_of_wind', '100m_v_component_of_wind', '10m_u_component_of_neutral_wind', '10m_u_component_of_wind', '10m_v_component_of_neutral_wind', '10m_v_component_of_wind', '10m_wind_gust_since_previous_post_processing', '2m_dewpoint_temperature', '2m_temperature', 'air_density_over_the_oceans', 'angle_of_sub_gridscale_orography', 'anisotropy_of_sub_gridscale_orography', 'benjamin_feir_index', 'boundary_layer_dissipation', 'boundary_layer_height', 'charnock', 'clear_sky_direct_solar_radiation_at_surface', 'cloud_base_height', 'coefficient_of_drag_with_waves', 'convective_available_potential_energy', 'convective_inhibition', 'convective_precipitation', 'convective_rain_rate', 'convective_snowfall', 'convective_snowfall_rate_water_equivalent', 'downward_uv_radiation_at_the_surface', 'duct_base_height', 'eastward_gravity_wave_surface_stress', 'eastward_turbulent_surface_stress', 'evaporation', 'forecast_albedo', 'forecast_logarithm_of_surface_roughness_for_heat', 'forecast_surfac

In [6]:
len(list(ds.data_vars))

273

## 1.2 Selecting Weather Features

### Core Features

#### Surface Temperature (°C)
- Variable: `2m_temperature`  
- Unit (ERA5): Kelvin (K) → converted to °C.

#### Relative Humidity (%)
Computed from:
- `2m_temperature` (K)  
- `2m_dewpoint_temperature` (K)

#### Wind Speed (m/s)
- `10m_u_component_of_wind` (m s⁻¹)  
- `10m_v_component_of_wind` (m s⁻¹)  
Wind speed = √(u² + v²)

#### Wind Direction (degrees)
Computed from u and v components:
\[
\text{direction} = \left( \text{atan2}(-u,\,-v) \cdot \frac{180}{\pi} \right) \bmod 360
\]
- 0° = North  
- 90° = East  
- 180° = South  
- 270° = West  
- Unit: degrees (°)

#### Total Precipitation (mm)
- Variable: `total_precipitation`  
- Unit (ERA5): meters of water equivalent (m) → converted to mm.

---

### Additional Weather Parameters

#### Wind Gusts (m/s)
- `instantaneous_10m_wind_gust` (m s⁻¹)  
- `10m_wind_gust_since_previous_post_processing` (m s⁻¹)

#### Cloud Cover (0–1)
- `total_cloud_cover`, `high_cloud_cover`, `medium_cloud_cover`, `low_cloud_cover`  
- Unit: fraction (0–1)

#### Snow and Ice (various)
- `snowfall` — m water equivalent  
- `snow_depth` — m  
- `snowmelt` — m water equivalent  
- `snow_density` — kg m⁻³

#### Precipitation Type
- `precipitation_type` — categorical (code table 4.201)

#### Surface Pressure (Pa)
- `surface_pressure`, `mean_sea_level_pressure`

#### Solar Radiation (J/m²)
- `surface_solar_radiation_downwards`  
- `clear_sky_direct_solar_radiation_at_surface`  
- `surface_net_solar_radiation`

#### Thermal Radiation (J/m²)
- `surface_thermal_radiation_downwards`  
- `surface_thermal_radiation_downward_clear_sky`

---

### Summary Table

| Category | Variables |
|---------|-----------|
| Temperature | `2m_temperature` |
| Humidity | `2m_temperature`, `2m_dewpoint_temperature` |
| Wind Speed | `10m_u_component_of_wind`, `10m_v_component_of_wind` |
| Wind Direction | computed from u,v |
| Precipitation | `total_precipitation` |
| Wind Gusts | `instantaneous_10m_wind_gust` |
| Cloud Cover | `total_cloud_cover` |
| Snow/Ice | `snowfall`, `snow_depth` |
| Pressure | `surface_pressure` |
| Solar Radiation | `surface_solar_radiation_downwards` |
| Thermal Radiation | `surface_thermal_radiation_downwards` |
| Precip Type | `precipitation_type` |

---

| Variable                                | **Original ERA5 Unit**   | **Final Unit (Stored in DuckDB)** | Notes                             |
| --------------------------------------- | ------------------------ | --------------------------------- | --------------------------------- |
| **2m_temperature**                      | Kelvin (K)               | **°C**                            | Converted: `T_C = T_K - 273.15`   |
| **2m_dewpoint_temperature**             | Kelvin (K)               | **°C**                            | Converted: `Td_C = Td_K - 273.15` |
| **10m_u_component_of_wind**             | m/s                      | **m/s**                           | No change                         |
| **10m_v_component_of_wind**             | m/s                      | **m/s**                           | No change                         |
| **wind_speed** (derived)                | m/s                      | **m/s**                           | Computed: `sqrt(u² + v²)`         |
| **wind_direction** (derived)            | degrees (meteorological) | **degrees**                       | `atan2(-u, -v) % 360`             |
| **total_precipitation**                 | meters (m) of water      | **mm**                            | `mm = m × 1000`                   |
| **snowfall**                            | meters (m)               | **mm**                            | Converted ×1000                   |
| **snow_depth**                          | meters (m)               | **mm**                            | Converted ×1000                   |
| **precipitation_type**                  | code                     | **code**                          | No change                         |
| **surface_pressure**                    | Pascals (Pa)             | **hPa (millibars)**               | `hPa = Pa / 100`                  |
| **surface_solar_radiation_downwards**   | Joules/m²                | **W/m²**                          | Divided by 3600 (per hour)        |
| **surface_thermal_radiation_downwards** | Joules/m²                | **W/m²**                          | Same conversion                   |
| **total_cloud_cover**                   | fraction (0–1)           | **%**                             | Multipled ×100                    |
| **instantaneous_10m_wind_gust**         | m/s                      | **m/s**                           | No change                         |
| **relative_humidity** (derived)         | % (computed)             | **%**                             | From T & Td using Magnus formula  |
| **longitude**                           | 0–360° or -180–180       | **-180 to 180**                   | Normalized for DB                 |
| **latitude**                            | degrees                  | **degrees**                       | No change                         |
| **time**                                | timestamp                | **timestamp (UTC)**               | No change                         |


---

### How Weather Parameters Affect a Regular Biker

| Parameter | Effect on a Regular Biker |
|----------|----------------------------|
| **Surface Temperature** | Very low or very high temperatures reduce comfort and discourage biking. Mild temperatures increase ridership. |
| **Relative Humidity** | High humidity increases perceived heat stress, making biking feel more tiring. Low humidity improves comfort. |
| **Wind Speed** | Strong winds make biking physically harder, reduce balance, and can prevent riders from starting a trip. |
| **Wind Direction** | Headwinds slow riders down and increase effort; tailwinds make riding easier; crosswinds can affect stability and safety. |
| **Wind Gusts** | Sudden strong gusts increase risk of losing control, especially for casual riders. |
| **Total Precipitation (Rain)** | Rain is one of the strongest deterrents; reduces visibility, comfort, and road safety, and leads to lower ridership. |
| **Snowfall / Snow Depth** | Snow and slush significantly reduce safety, traction, and bike availability; bikers tend to avoid riding altogether. |
| **Snowmelt / Ice** | Melting snow or ice creates slippery surfaces, increasing risk of accidents. |
| **Cloud Cover** | Heavy cloud cover reduces daylight and visibility; moderate cloud cover has minor impact. |
| **Surface Pressure** | Indirect effect; low pressure often accompanies storms or poor weather, reducing willingness to bike. |
| **Solar Radiation** | High solar radiation increases heat load and discomfort; moderate levels improve visibility and warmth. |
| **Thermal Radiation** | Influences perceived outdoor warmth, especially during early morning or evening trips. |
| **Precipitation Type** | Frozen precipitation (sleet, freezing rain) creates hazardous surfaces and strongly discourages biking. |


In [8]:
era_vars = [
    "2m_temperature",
    "2m_dewpoint_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "total_precipitation",
    "total_cloud_cover",
    "instantaneous_10m_wind_gust",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "surface_pressure",
    "precipitation_type",
    "snowfall",
    "snow_depth",
]

# Filter to only the variables we care about
ds = ds[era_vars]

## 2. Get Raw Weather Data for Boston between 2010 and 2025
stored in Zarr format

In [ ]:
DB_PATH = "bluebikes_2015_2025.duckdb"
PARQUET_PATH = "weather_boston_era5.parquet"
ERA5_ZARR = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"

ERA_VARS = [
    "2m_temperature", "2m_dewpoint_temperature",
    "10m_u_component_of_wind", "10m_v_component_of_wind",
    "total_precipitation", "total_cloud_cover",
    "instantaneous_10m_wind_gust",
    "surface_solar_radiation_downwards",
    "surface_thermal_radiation_downwards",
    "surface_pressure", "precipitation_type",
    "snowfall", "snow_depth",
]

# 1. Bounds
log.info("Connecting to DuckDB...")
con = duckdb.connect(DB_PATH)

log.info("Reading trip bounds...")
bounds = con.execute("""
    SELECT
        min(start_lat), max(start_lat),
        min(start_lng), max(start_lng),
        min(started_at), max(started_at)
    FROM trips
""").fetchone()

min_lat, max_lat, min_lon, max_lon, t0, t1 = bounds

lat_pad = 0.5
lon_pad = 0.5

lat_min = min_lat - lat_pad
lat_max = max_lat + lat_pad
lon_min = min_lon - lon_pad
lon_max = max_lon + lon_pad

t_min = pd.to_datetime(t0).tz_localize(None)
t_max = pd.to_datetime(t1).tz_localize(None)

log.info(f"Lat range: {lat_min} → {lat_max}")
log.info(f"Lon range: {lon_min} → {lon_max}")
log.info(f"Time range: {t_min} → {t_max}")

# 2. ERA5 subset
log.info("Opening ERA5 Zarr...")
ds = xr.open_zarr(ERA5_ZARR, chunks={"time": 48}, consolidated=True)
ds = ds[ERA_VARS]

log.info("Subsetting ERA5...")
# Detect latitude order
lat0 = float(ds.latitude[0])
latN = float(ds.latitude[-1])
if lat0 > latN:
    # descending
    lat_slice = slice(lat_max, lat_min)
else:
    # ascending
    lat_slice = slice(lat_min, lat_max)

# Convert lon from [-180, 180] to [0, 360)
def to_360(x):
    return (x + 360) % 360

lon_min_360 = to_360(lon_min)
lon_max_360 = to_360(lon_max)

# For Boston region, lon_min_360 < lon_max_360, so we can use a simple slice
lon_slice = slice(lon_min_360, lon_max_360)

ds_sub = ds.sel(
    time=slice(t_min, t_max),
    latitude=lat_slice,
    longitude=lon_slice,
)
log.info("Converting to Zarr...")
ds_sub.to_zarr(
    "weather_boston_era5.zarr",
    mode="w",
    consolidated=True,
    zarr_version=2,
)
log.info("done")

2025-11-15 04:55:36,875 [INFO] Connecting to DuckDB...
2025-11-15 04:55:36,884 [INFO] Reading trip bounds...
2025-11-15 04:55:36,947 [INFO] Lat range: 41.63792965 → 46.005086
2025-11-15 04:55:36,947 [INFO] Lon range: -74.066921 → -70.27669668333333
2025-11-15 04:55:36,947 [INFO] Time range: 2015-01-01 00:21:44 → 2025-10-31 23:56:01.017000
2025-11-15 04:55:36,947 [INFO] Opening ERA5 Zarr...
2025-11-15 04:55:38,447 [INFO] Subsetting ERA5...
2025-11-15 04:55:38,509 [INFO] Converting to Zarr...
/tmp/ipykernel_4515/3937214709.py:116: FutureWarning: zarr_version is deprecated, use zarr_format
  ds_sub.to_zarr(


## 3. Merge Weather Dataset with Trip Dataset
- Calculate Derived fields (Wind Speed, Wind Direction, and Relative Humadity)
- Export to DuckDB

Ask Zarr file for the interpolated weather corresponding to each trip based on its timestamp and spatial coordinates. The weather dataset is hourly and gridded (each hour contains 18×15 = 270 grid cells). When mapping raw weather data to trips, multiple trips within the same hour may receive identical weather values, even if they occur at different minutes or slightly different locations. For example, a trip starting at the beginning of the hour and another starting near the end of the hour - both located near the same grid cell - would end up with the same weather record. This is why we use the interpolated version.

### 3.1 Merge without Interpolation

#### Create `weather` table (raw weather data, from zarr to duckdb)

In [21]:
ds = xr.open_zarr('weather_boston_era5.zarr', chunks={"time": 48}, consolidated=True)

log.info("Connecting to DuckDB...")
con = duckdb.connect("bluebikes_2015_2025.duckdb")

# convert lon from [0, 360) → [-180, 180)
def to_minus180_180(x):
    return ((x + 180) % 360) - 180

daterange = pd.date_range("2015-01-01", "2026-01-01", freq="MS")

table_created = False
startmonth = None

for index, endmonth in enumerate(daterange):
    if index == 0:
        con.execute("DROP TABLE IF EXISTS weather")
        startmonth = endmonth
        continue

    t_min = pd.to_datetime(startmonth).tz_localize(None)
    t_max = pd.to_datetime(endmonth - timedelta(seconds=1)).tz_localize(None)
    log.info(f"Time range: {t_min} → {t_max}")

    # load ERA5 for this month
    ds_sub = ds.sel(time=slice(t_min, t_max))
    df = ds_sub.to_dataframe().reset_index()

    if df.empty:
        log.info("No ERA5 data — skipping.")
        startmonth = endmonth
        continue

    # Kelvin → Celsius
    df["2m_temperature"] = df["2m_temperature"] - 273.15
    df["2m_dewpoint_temperature"] = df["2m_dewpoint_temperature"] - 273.15

    # Precip/snow meters → mm
    df["total_precipitation"] = df["total_precipitation"] * 1000.0
    df["snowfall"] = df["snowfall"] * 1000.0
    df["snow_depth"] = df["snow_depth"] * 1000.0

    # Radiation J/m² → W/m²
    df["surface_solar_radiation_downwards"] = df["surface_solar_radiation_downwards"] / 3600.0
    df["surface_thermal_radiation_downwards"] = df["surface_thermal_radiation_downwards"] / 3600.0

    # Pressure Pa → hPa
    df["surface_pressure"] = df["surface_pressure"] / 100.0

    # Cloud cover fraction → %
    df["total_cloud_cover"] = df["total_cloud_cover"] * 100.0

    # Derived wind speed + direction
    u = df["10m_u_component_of_wind"]
    v = df["10m_v_component_of_wind"]
    df["wind_speed"] = (u * u + v * v) ** 0.5
    df["wind_direction"] = (np.degrees(np.arctan2(-u, -v)) % 360)

    # Derived relative humidity %
    T = df["2m_temperature"]
    Td = df["2m_dewpoint_temperature"]
    es = 6.112 * np.exp((17.67 * T) / (T + 243.5))
    e = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))
    df["relative_humidity"] = 100.0 * (e / es)

    # Normalize longitude
    df["longitude"] = to_minus180_180(df["longitude"])

    log.info(f"ERA5 rows for {startmonth:%Y-%m}: {len(df):,}")

    con.register("tmp_weather", df)

    if not table_created:
        con.execute("CREATE TABLE weather AS SELECT * FROM tmp_weather")
        table_created = True
    else:
        con.execute("INSERT INTO weather SELECT * FROM tmp_weather")

    con.unregister("tmp_weather")

    startmonth = endmonth

log.info("Done building 'weather' table.")
con.close()



2025-11-17 09:06:55,591 [INFO] Connecting to DuckDB...
2025-11-17 09:06:55,647 [INFO] Time range: 2015-01-01 00:00:00 → 2015-01-31 23:59:59
2025-11-17 09:06:55,647 [INFO] Time range: 2015-01-01 00:00:00 → 2015-01-31 23:59:59
2025-11-17 09:07:03,711 [INFO] ERA5 rows for 2015-01: 200,610
2025-11-17 09:07:03,711 [INFO] ERA5 rows for 2015-01: 200,610
2025-11-17 09:07:03,796 [INFO] Time range: 2015-02-01 00:00:00 → 2015-02-28 23:59:59
2025-11-17 09:07:03,796 [INFO] Time range: 2015-02-01 00:00:00 → 2015-02-28 23:59:59
2025-11-17 09:07:10,310 [INFO] ERA5 rows for 2015-02: 181,440
2025-11-17 09:07:10,310 [INFO] ERA5 rows for 2015-02: 181,440
2025-11-17 09:07:10,374 [INFO] Time range: 2015-03-01 00:00:00 → 2015-03-31 23:59:59
2025-11-17 09:07:10,374 [INFO] Time range: 2015-03-01 00:00:00 → 2015-03-31 23:59:59
2025-11-17 09:07:17,529 [INFO] ERA5 rows for 2015-03: 200,880
2025-11-17 09:07:17,529 [INFO] ERA5 rows for 2015-03: 200,880
2025-11-17 09:07:17,601 [INFO] Time range: 2015-04-01 00:00:00 

#### Join `trips` and `weather` tables into a new table `trips_weather_no_interpolation`

In [25]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")

con.execute("DROP TABLE IF EXISTS trips_weather_no_interpolation")

# Get month starts from trips
months_df = con.execute("""
    SELECT date_trunc('month', started_at) AS month_start
    FROM trips
    GROUP BY month_start
    ORDER BY month_start
""").fetchdf()

month_starts = pd.to_datetime(months_df["month_start"])

created = False

for i, month_start in enumerate(month_starts):
    month_start = pd.to_datetime(month_start)
    month_end = month_start + MonthBegin(1)

    print(f"Processing {month_start.date()} → {month_end.date()}")

    # Just to see how many trips this month
    n_trips = con.execute(
        """
        SELECT COUNT(*) AS cnt
        FROM trips
        WHERE started_at >= ? AND started_at < ?
        """,
        [month_start, month_end],
    ).fetchone()[0]
    print(f"  Trips in this month: {n_trips:,}")
    if n_trips == 0:
        continue

    if not created:
        # First month: create table
        con.execute("""
            CREATE TABLE trips_weather_no_interpolation AS
            SELECT
                t.*,
                w.time       AS weather_time,
                w.latitude   AS weather_lat,
                w.longitude  AS weather_lon,
                w."10m_u_component_of_wind",
                w."10m_v_component_of_wind",
                w."2m_dewpoint_temperature",
                w."2m_temperature",
                w.instantaneous_10m_wind_gust,
                w.precipitation_type,
                w.snow_depth,
                w.snowfall,
                w.surface_pressure,
                w.surface_solar_radiation_downwards,
                w.surface_thermal_radiation_downwards,
                w.total_cloud_cover,
                w.total_precipitation,
                w.wind_speed,
                w.wind_direction,
                w.relative_humidity
            FROM trips t
            LEFT JOIN weather w
              ON date_trunc('hour', t.started_at + INTERVAL 30 MINUTE) = w.time
             AND round(t.start_lat * 4) / 4.0 = w.latitude
             AND round(t.start_lng * 4) / 4.0 = w.longitude
            WHERE t.started_at >= ? AND t.started_at < ?
        """, [month_start, month_end])
        created = True
    else:
        # Subsequent months: append
        con.execute("""
            INSERT INTO trips_weather_no_interpolation
            SELECT
                t.*,
                w.time       AS weather_time,
                w.latitude   AS weather_lat,
                w.longitude  AS weather_lon,
                w."10m_u_component_of_wind",
                w."10m_v_component_of_wind",
                w."2m_dewpoint_temperature",
                w."2m_temperature",
                w.instantaneous_10m_wind_gust,
                w.precipitation_type,
                w.snow_depth,
                w.snowfall,
                w.surface_pressure,
                w.surface_solar_radiation_downwards,
                w.surface_thermal_radiation_downwards,
                w.total_cloud_cover,
                w.total_precipitation,
                w.wind_speed,
                w.wind_direction,
                w.relative_humidity
            FROM trips t
            LEFT JOIN weather w
              ON date_trunc('hour', t.started_at + INTERVAL 30 MINUTE) = w.time
             AND round(t.start_lat * 4) / 4.0 = w.latitude
             AND round(t.start_lng * 4) / 4.0 = w.longitude
            WHERE t.started_at >= ? AND t.started_at < ?
        """, [month_start, month_end])

    # Optional: count how many rows we just added
    n_added = con.execute("""
        SELECT COUNT(*) FROM trips_weather_no_interpolation
        WHERE started_at >= ? AND started_at < ?
    """, [month_start, month_end]).fetchone()[0]
    print(f"  Rows in trips_weather_no_interpolation for this month: {n_added:,}")

con.close()
print("Done building trips_weather_no_interpolation.")


Processing 2015-01-01 → 2015-02-01
  Trips in this month: 7,840
  Rows in trips_weather_no_interpolation for this month: 7,840
Processing 2015-02-01 → 2015-03-01
  Trips in this month: 4,020
  Rows in trips_weather_no_interpolation for this month: 4,020
Processing 2015-03-01 → 2015-04-01
  Trips in this month: 11,270
  Rows in trips_weather_no_interpolation for this month: 11,270
Processing 2015-04-01 → 2015-05-01
  Trips in this month: 55,843
  Rows in trips_weather_no_interpolation for this month: 55,843
Processing 2015-05-01 → 2015-06-01
  Trips in this month: 149,143
  Rows in trips_weather_no_interpolation for this month: 55,843
Processing 2015-05-01 → 2015-06-01
  Trips in this month: 149,143
  Rows in trips_weather_no_interpolation for this month: 149,143
Processing 2015-06-01 → 2015-07-01
  Trips in this month: 144,232
  Rows in trips_weather_no_interpolation for this month: 149,143
Processing 2015-06-01 → 2015-07-01
  Trips in this month: 144,232
  Rows in trips_weather_no_int

### 3.2 Merge with Interpolation

#### Building `weather_at_trips` table

In [14]:
# 1) Open local ERA5 subset
ds = xr.open_zarr("weather_boston_era5.zarr", consolidated=True)

# Detect longitude convention once
lon_min = float(ds.longitude.min())
lon_max = float(ds.longitude.max())
if lon_min >= 0 and lon_max > 180:
    # dataset uses [0, 360)
    def to_ds_lon(lon_series):
        return (lon_series + 360.0) % 360.0
else:
    # dataset uses [-180, 180]
    def to_ds_lon(lon_series):
        return lon_series

# 2) Connect to DuckDB
con = duckdb.connect("bluebikes_2015_2025.duckdb")
con.execute("DROP TABLE IF EXISTS weather_at_trips")

# 3) Get all months present in trip table
months_df = con.execute("""
    SELECT date_trunc('month', started_at) AS month_start
    FROM trips
    GROUP BY month_start
    ORDER BY month_start
""").fetchdf()

first_batch = True

for month_start in pd.to_datetime(months_df["month_start"]):
    month_end = month_start + MonthBegin(1)
    print(f"Processing {month_start.date()} → {month_end.date()}")

    # 3.1 Get trips for this month
    trips = con.execute(
        """
        SELECT
            _id,
            started_at,
            ended_at,
            start_lat, start_lng,
            end_lat,   end_lng
        FROM trips
        WHERE started_at >= ? AND started_at < ?
        ORDER BY started_at
        """,
        [month_start, month_end],
    ).fetchdf()

    if trips.empty:
        print("  No trips in this month, skipping.")
        continue

    # 3.2 Compute midpoints (time + location)
    start_t = pd.to_datetime(trips["started_at"])
    end_t   = pd.to_datetime(trips["ended_at"])
    mid_times = (start_t + (end_t - start_t) / 2).dt.tz_localize(None)

    mid_lats = 0.5 * (trips["start_lat"].astype(float) + trips["end_lat"].astype(float))
    mid_lons = 0.5 * (trips["start_lng"].astype(float) + trips["end_lng"].astype(float))
    mid_lons_ds = to_ds_lon(mid_lons)

    # 3.3 Subset ERA5 in time only (Zarr is already Boston subset)
    t_min, t_max = mid_times.min(), mid_times.max()
    ds_month = ds.sel(time=slice(t_min, t_max))

    if ds_month.sizes.get("time", 0) < 1:
        print("  ERA5 has no data in this time range, skipping.")
        continue

    # 3.4 Interpolate at midpoints
    points = ds_month.interp(
        time=("points", mid_times.to_numpy()),
        latitude=("points", mid_lats.to_numpy()),
        longitude=("points", mid_lons_ds.to_numpy()),
        method="linear",
    )

    wdf = points.to_dataframe().reset_index(drop=True)

    # -------------------------------------------------
    # 3.5 Override raw units with paper-friendly units
    # -------------------------------------------------

    # Temperature: K → °C
    wdf["2m_temperature"] = wdf["2m_temperature"] - 273.15
    wdf["2m_dewpoint_temperature"] = wdf["2m_dewpoint_temperature"] - 273.15

    # Precipitation & snow: m → mm
    wdf["total_precipitation"] = wdf["total_precipitation"] * 1000.0
    wdf["snowfall"] = wdf["snowfall"] * 1000.0
    wdf["snow_depth"] = wdf["snow_depth"] * 1000.0

    # Radiation: J/m² → W/m²
    wdf["surface_solar_radiation_downwards"] = (
        wdf["surface_solar_radiation_downwards"] / 3600.0
    )
    wdf["surface_thermal_radiation_downwards"] = (
        wdf["surface_thermal_radiation_downwards"] / 3600.0
    )

    # Surface pressure: Pa → hPa
    wdf["surface_pressure"] = wdf["surface_pressure"] / 100.0

    # Cloud cover: fraction → %
    wdf["total_cloud_cover"] = wdf["total_cloud_cover"] * 100.0

    # Wind derived variables (keep u, v in m/s)
    u = wdf["10m_u_component_of_wind"]
    v = wdf["10m_v_component_of_wind"]
    wdf["wind_speed"] = np.sqrt(u * u + v * v)  # m/s
    wdf["wind_direction"] = np.degrees(np.arctan2(-u, -v)) % 360  # degrees

    # Relative humidity (%) using °C
    T  = wdf["2m_temperature"]
    Td = wdf["2m_dewpoint_temperature"]
    es = 6.112 * np.exp((17.67 * T)  / (T  + 243.5))
    e  = 6.112 * np.exp((17.67 * Td) / (Td + 243.5))
    wdf["relative_humidity"] = 100.0 * (e / es)

    # If ds is 0–360 and you prefer storing [-180, 180]
    if lon_min >= 0 and lon_max > 180:
        wdf["longitude"] = ((wdf["longitude"] + 180.0) % 360.0) - 180.0

    # 3.6 Combine _id + interpolated weather
    weather_trips = pd.concat(
        [trips[["_id"]].reset_index(drop=True),
         wdf.reset_index(drop=True)],
        axis=1,
    )

    print(f"  Interpolated rows: {len(weather_trips):,}")

    # 3.7 Append to DuckDB
    con.register("tmp_weather_trips", weather_trips)

    if first_batch:
        con.execute("CREATE TABLE weather_at_trips AS SELECT * FROM tmp_weather_trips")
        first_batch = False
    else:
        con.execute("INSERT INTO weather_at_trips SELECT * FROM tmp_weather_trips")

    con.unregister("tmp_weather_trips")

con.close()
print("Done building weather_at_trips.")



Processing 2015-01-01 → 2015-02-01
  Interpolated rows: 7,840
Processing 2015-02-01 → 2015-03-01
  Interpolated rows: 7,840
Processing 2015-02-01 → 2015-03-01
  Interpolated rows: 4,020
Processing 2015-03-01 → 2015-04-01
  Interpolated rows: 4,020
Processing 2015-03-01 → 2015-04-01
  Interpolated rows: 11,270
Processing 2015-04-01 → 2015-05-01
  Interpolated rows: 11,270
Processing 2015-04-01 → 2015-05-01
  Interpolated rows: 55,843
Processing 2015-05-01 → 2015-06-01
  Interpolated rows: 55,843
Processing 2015-05-01 → 2015-06-01
  Interpolated rows: 149,143
Processing 2015-06-01 → 2015-07-01
  Interpolated rows: 149,143
Processing 2015-06-01 → 2015-07-01
  Interpolated rows: 144,232
Processing 2015-07-01 → 2015-08-01
  Interpolated rows: 144,232
Processing 2015-07-01 → 2015-08-01
  Interpolated rows: 175,321
  Interpolated rows: 175,321
Processing 2015-08-01 → 2015-09-01
Processing 2015-08-01 → 2015-09-01
  Interpolated rows: 160,065
  Interpolated rows: 160,065
Processing 2015-09-01 →

#### Join `trips` and `weather_at_trips` tables into a new table `trips_weather_interpolated`

In [17]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")

con.execute("DROP TABLE IF EXISTS trips_weather_interpolated")

con.execute("""
CREATE TABLE trips_weather_interpolated AS
SELECT
    t.*,
    w.time      AS weather_time,
    w.latitude  AS weather_lat,
    w.longitude AS weather_lon,
    w."10m_u_component_of_wind",
    w."10m_v_component_of_wind",
    w."2m_dewpoint_temperature",
    w."2m_temperature",
    w.instantaneous_10m_wind_gust,
    w.precipitation_type,
    w.snow_depth,
    w.snowfall,
    w.surface_pressure,
    w.surface_solar_radiation_downwards,
    w.surface_thermal_radiation_downwards,
    w.total_cloud_cover,
    w.total_precipitation,
    w.wind_speed,
    w.wind_direction,
    w.relative_humidity
FROM trips t
LEFT JOIN weather_at_trips w USING (_id);
""")

con.close()
print("Created trips_weather.")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Created trips_weather.


## 4. Queries

### `trips` table

In [36]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""
    SELECT *
    FROM trips
    WHERE started_at >= '2015-01-01'
      AND started_at <  '2015-02-01';
  """).df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [15]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""SELECT count(*) from trips""").df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


### `weather` table

In [ ]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""
    SELECT *
    FROM weather
    WHERE time >= '2015-01-01'
      AND time <  '2015-02-01';
  """).df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [32]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""SELECT count(*) from weather""").df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


### `trips_weather_no_interpolation` table

In [10]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""
    SELECT wind_speed
    FROM trips_weather_no_interpolation
    WHERE started_at >= '2015-01-01'
      AND started_at <  '2015-02-01'
                 order by started_at asc;
  """).df()
con.close()
df

CatalogException: Catalog Error: Table with name trips_weather_no_interpolation does not exist!
Did you mean "trips_weather_interpolated"?

LINE 3:     FROM trips_weather_no_interpolation
                 ^

In [ ]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""SELECT count(*) from trips_weather_no_interpolation""").df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


### `weather_at_trips` table

In [30]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""
    SELECT *
    FROM weather_at_trips
    WHERE time >= '2015-01-01'
      AND time <  '2015-03-01';
  """).df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


In [16]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""SELECT count(*) from weather_at_trips""").df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


### `trips_weather_interpolated` table

In [15]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""
    SELECT *
    FROM trips_weather_interpolated
    WHERE weather_time >= '2015-01-01'
      AND weather_time <  '2015-03-01';
  """).df()
con.close()
df.to_csv('test.csv')

In [16]:
df = pd.read_csv('test.csv')

In [9]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
df = con.execute("""SELECT count(*) from trips_weather_interpolated""").df()
con.close()
df

Loading ITables v2.5.2 from the init_notebook_mode cell... (need help?)


## 5. Cleaning Dataset (keep only `trips_weather_interpolated` table)

In [ ]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
con.execute("""
    -- Truncate tables first
    -- WHY truncate before drop? To avoid issues db size and vacuuming
    TRUNCATE trips;
    TRUNCATE trips_weather_no_interpolation;
    TRUNCATE weather;
    TRUNCATE weather_at_trips;

    -- Create backup copy
    CREATE TABLE trips_weather_interpolated2 AS
    SELECT *
    FROM trips_weather_interpolated;

    -- Drop all other tables
    DROP TABLE trips;
    DROP TABLE trips_weather_no_interpolation;
    DROP TABLE weather;
    DROP TABLE weather_at_trips;
    DROP TABLE trips_weather_interpolated;

    -- Rename the remaining table
    ALTER TABLE trips_weather_interpolated2
    RENAME TO trips_weather_interpolated;
    
    -- Clean up rows with no weather data (first hour of 2015)
    DELETE FROM trips_weather_interpolated WHERE _id IN (1,2,3,4);
""")

con.close()


## 6. Export to Parquet

In [2]:
con = duckdb.connect("bluebikes_2015_2025.duckdb")
con.execute("""COPY trips_weather_interpolated TO 'bluebikes_2015_2025.parquet' (FORMAT PARQUET);""")
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))